In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames


## INFORMACION DATASET (FILM BETCHDEL TEST)

El Test de Bechdel evalúa si una obra de ficción incluye a al menos dos mujeres que hablen entre ellas de algo que no sea un hombre, como una forma de medir la representación femenina en los medios.

Resultados TEST: 

- ok: Películas o episodios que han pasado el Test de Bechdel, es decir, incluyen a dos mujeres que hablan entre ellas de algo que no es un   hombre.

- notalk: Películas o episodios que no pasan el test porque no hay ninguna conversación entre mujeres, o simplemente no hay personajes femeninos hablando en absoluto.

- men: Películas o episodios pueden haber tenido conversaciones entre mujeres, pero dichas conversaciones están centradas únicamente en hombres, por lo tanto, no pasan el Test de Bechdel.

- dubious: presentan situaciones ambiguas o difíciles de clasificar, como diálogos mínimos entre mujeres que no dejan claro si cumplen con los criterios del test o que podrían no ajustarse claramente a los parámetros establecidos.

- nowomen: 141 películas o episodios que no tienen personajes femeninos, lo que automáticamente las descalifica del Test de Bechdel.



En el contexto de análisis de películas, las columnas domgross y intgross se refieren a los ingresos de taquilla en diferentes mercados:

- domgross: Hace referencia a los ingresos brutos en el mercado doméstico, es decir, los ingresos generados por la película en su país de origen, que normalmente sería Estados Unidos si estás analizando películas de Hollywood. "Dom" proviene de "domestic" (doméstico), y "gross" indica ingresos brutos, que es el total de dinero recaudado antes de descontar gastos como la distribución y la publicidad.

- intgross: Se refiere a los ingresos brutos internacionales, es decir, la cantidad de dinero que la película ha generado en mercados fuera de su país de origen. "Int" proviene de "international" (internacional). Este valor refleja los ingresos de taquilla obtenidos en otros países.

La razón por la cual tienes columnas como domgross e intgross por un lado, y domgross_2013$ e intgross_2013$ por otro, es que las segundas están ajustadas por inflación para hacer las cifras comparables en términos reales.

El problema con estas cifras es que no toman en cuenta la inflación. Una película que ganó $100 millones en 1990 no puede ser directamente comparada con una que ganó $100 millones en 2020, ya que el valor del dinero cambia con el tiempo.

- domgross_2013$ e intgross_2013$:

Estas columnas muestran los ingresos ajustados por inflación hasta el año 2013, lo que significa que los valores de las películas más antiguas han sido ajustados para reflejar lo que ese dinero valdría en 2013.
Por ejemplo, si una película de 1990 ganó $100 millones, esa cifra puede haber sido ajustada a $180 millones en dólares de 2013, porque la inflación hizo que el valor del dinero disminuyera con el tiempo.
El ajuste por inflación es útil para comparar el éxito financiero de películas de diferentes años de manera más justa.



In [2]:
# Abrimos csv con la información relativa al test de betchdel

df1 = pd.read_csv('movies_bechdel.csv', index_col=0)

df1.info()

# con el método info vemos que no hay muchos nulos en ninguna de las columnas.
# el tipo de dato parece correcto, datos numericos con int o float, y datos categoricos con de tipo object.

<class 'pandas.core.frame.DataFrame'>
Index: 1794 entries, 2013 to 1970
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   imdb            1794 non-null   object 
 1   title           1794 non-null   object 
 2   test            1794 non-null   object 
 3   clean_test      1794 non-null   object 
 4   binary          1794 non-null   object 
 5   budget          1794 non-null   int64  
 6   domgross        1777 non-null   float64
 7   intgross        1783 non-null   float64
 8   code            1794 non-null   object 
 9   budget_2013$    1794 non-null   int64  
 10  domgross_2013$  1776 non-null   float64
 11  intgross_2013$  1783 non-null   float64
 12  period code     1615 non-null   float64
 13  decade code     1615 non-null   float64
dtypes: float64(6), int64(2), object(6)
memory usage: 210.2+ KB


In [3]:
df1.describe(include=object)
# vemos que los identificadores imdb con identificadores unicos, el recuento de unique

,imdb,title,test,clean_test,binary,code
count,1794,1794,1794,1794,1794,1794
unique,1794,1768,10,5,2,85
top,tt1711425,Pride and Prejudice,ok,ok,FAIL,2009FAIL
freq,1,3,696,803,991,82


In [5]:
# Abrimos el csv que contiene información de las películas, pero no tiene información del test de Betchdel

df2 = pd.read_csv('clean_completo.csv')

df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105425 entries, 0 to 105424
Data columns (total 20 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   adult                       105425 non-null  object 
 1   budget                      105425 non-null  object 
 2   id                          105425 non-null  object 
 3   imdb_id                     105404 non-null  object 
 4   original_language           105408 non-null  object 
 5   original_title              105425 non-null  object 
 6   overview                    104403 non-null  object 
 7   popularity                  105408 non-null  object 
 8   release_date                105389 non-null  object 
 9   revenue                     105404 non-null  float64
 10  runtime                     105184 non-null  float64
 11  status                      105353 non-null  object 
 12  title                       105404 non-null  object 
 13  vote_average  

C:\Users\Laura Romero\AppData\Local\Temp\ipykernel_9340\898791022.py:3: DtypeWarning: Columns (0,1,2,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('clean_completo.csv')


In [7]:
# unificamos los dos dataframes usando el elemento común que es imdb/imdb_id, para tener un csv con información más detallada
# de las películas que tienen índice de betchdel. 

df_merge = pd.merge(df1, df2, left_on='imdb', right_on='imdb_id')

In [8]:
# Eliminamos las columnas que tienen información redundante. 

df_merge.drop(columns=['title_y', 'imdb_id', 'budget_y'], inplace=True)

In [9]:
# modificamos el nombre de algunas columnas para que sea más claro. 
diccionario = {'title_x' : 'title', 'budget_x': 'budget'}

df_merge.rename(columns= diccionario, inplace=True)

In [14]:
len(df_merge['imdb'].unique())

1776

In [15]:
# Guardamos en un csv
df_merge.to_csv('film_betchdel_test.csv', index=False)